In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l

In [2]:
def dropout(X,drop_prob):
	X = X.float()
	assert 0 <= drop_prob <= 1
	keep_prob = 1 - drop_prob
	if keep_prob == 0:
		return torch.zeros_like(X)
	mask = (torch.randn(X.shape) < keep_prob).float()
	
	return mask * X / keep_prob

In [3]:
#test
X = torch.arange(16).view(2,8)
dropout(X,0)


tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  0.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])

In [4]:
dropout(X,0.5)


tensor([[ 0.,  2.,  0.,  6.,  8.,  0., 12., 14.],
        [16., 18., 20., 22., 24., 26.,  0., 30.]])

In [5]:
dropout(X,1.0)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
#定义模型参数
num_inputs,num_outputs,num_hiddens1,num_hiddens2 = 784,10,256,256
W1 = torch.tensor(np.random.normal(0,0.01,size=(num_inputs,num_hiddens1)),
	dtype = torch.float,requires_grad=True)
b1 = torch.zeros(num_hiddens1,requires_grad=True)

W2 = torch.tensor(np.random.normal(0,0.01,size=(num_hiddens1,num_hiddens2)),
    dtype = torch.float,requires_grad=True)
b2 = torch.zeros(num_hiddens2,requires_grad=True)

W3 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens2,num_outputs)), 
    dtype=torch.float, requires_grad=True)
b3 = torch.zeros(num_outputs,requires_grad=True)

params = [W1,b1,W2,b2,W3,b3]

In [7]:
#定义模型
drop_prob1,drop_prob2 = 0.2,0.5

def net(X,is_training=True):
	X = X.view(-1,num_inputs)
	H1 = (torch.matmul(X,W1) + b1).relu()
	if is_training:
		H1 = dropout(H1,drop_prob1)
	H2 = (torch.matmul(H1,W2) + b2).relu()
	if is_training:
		H2 = dropout(H2,drop_prob2)
	return torch.matmul(H2,W3) + b3

In [8]:
#训练和测试模型
num_epochs,lr,batch_size = 5,100.0,256
loss = torch.nn.CrossEntropyLoss()
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params,lr)

epoch 1, loss 0.0045, train acc 0.548, test acc 0.659
epoch 2, loss 0.0023, train acc 0.780, test acc 0.775
epoch 3, loss 0.0019, train acc 0.823, test acc 0.815
epoch 4, loss 0.0017, train acc 0.839, test acc 0.847
epoch 5, loss 0.0016, train acc 0.851, test acc 0.847
